In [1]:
import pandas as pd
from tqdm import tqdm
from pyarrow import csv
import pyarrow as pa

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from catboost import CatBoostClassifier
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler, StandardScaler
from sklearn.metrics import auc, accuracy_score, precision_score, recall_score, roc_curve, precision_recall_curve

import pandas as pd
from tqdm import tqdm
from pyarrow import csv
import pyarrow as pa

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from catboost import CatBoostClassifier
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

In [2]:
def get_data(filename, scaler=None, useScaler=False):
    df = pd.read_csv(filename)
    Y = df.Label
    df = df.drop(['Unnamed: 0', 'Label'], axis=1)

    df['Attack'] = df['Attack'].astype("string")

    X_train, X_test, y_train, y_test = train_test_split(df, Y, test_size=0.3,stratify=df.Attack, random_state=42)
    del df
    del Y

    attack_train = X_train['Attack']
    X_train = X_train.drop('Attack', axis=1)
    attack_test = X_test['Attack']
    X_test = X_test.drop('Attack', axis=1)

    X_train = X_train.drop(['SimillarHTTP'], axis=1)
    X_train[X_train['Flow Bytes/s'] == np.inf] = 0
    X_train[X_train[' Flow Packets/s'] == np.inf] = 0

    X_test = X_test.drop(['SimillarHTTP'], axis=1)
    X_test[X_test['Flow Bytes/s'] == np.inf] = 0
    X_test[X_test[' Flow Packets/s'] == np.inf] = 0

    
    if useScaler:
        X_train = scaler.transform(X_train)
    else:
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test, attack_train, attack_test, scaler

In [3]:
X_train, X_test, y_train, y_test, attack_train, attack_test, scaler = get_data('triple.csv')

/tmp/ipykernel_200748/3331398274.py:2: DtypeWarning: Columns (62,109) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [4]:
save(scaler, "scaler.pckl")

'scaler.pckl)' was not found in history, as a file, url, nor in the user namespace.


In [20]:
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    
class modelDNN(nn.Module):
    def __init__(self):
        super(modelDNN, self).__init__()        
        self.layer_1 = nn.Linear(111, 4096) 
        self.layer_2 = nn.Linear(4096, 2048)
        self.layer_3 = nn.Linear(2048, 64) 
        self.layer_4 = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(4096)
        self.batchnorm2 = nn.BatchNorm1d(2048)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.dropout(x)
        x = self.layer_4(x)
        
        return x
    
def binary_acc(y_pred, y_test):
    print(y_pred)
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    print(y_pred_tag)

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    print(correct_results_sum)
    acc = correct_results_sum/y_test.shape[0]
    print(acc)
    acc *= 100
    print("---------------------------------")
    
    return acc

In [6]:
device = torch.device("cuda:0")

In [7]:
train_data = TrainData(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
test_data = TestData(torch.FloatTensor(X_test))

In [8]:
model = modelDNN().to(device)

In [9]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [10]:
        
train_loader = DataLoader(dataset=train_data, batch_size=512, shuffle=True)
epochs = 20

In [11]:
del X_train, attack_train, attack_test, scaler, y_train

In [12]:
for i in range(8):
    print("y_pred_"+str(i))

y_pred_0
y_pred_1
y_pred_2
y_pred_3
y_pred_4
y_pred_5
y_pred_6
y_pred_7


In [13]:
import pickle

def save(data, filename):
    file = open(filename, 'wb')
    pickle.dump(data, file)
    file.close()

In [16]:
for e in range(epochs):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()

        y_pred = model(X_batch)

        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    model.eval()
    
    for i in range(8):
        del y_pred
        y_pred = model(torch.Tensor(X_test[i*100000 : min(i*100000 + 100000, X_test.shape[0])]).to(device))
        save(y_pred, "y_pred_"+str(i))
        test_acc = binary_acc(y_pred, torch.Tensor(np.array(y_test[i*100000 : min(i*100000 + 100000, y_test.shape[0])])).to(device).unsqueeze(1)) 
        print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Train Acc: {epoch_acc/len(train_loader):.3f} | Test Acc: {test_acc:.3f}')
    print("-----------------------------------------------------------------------------------------------------------------------\n")

Epoch 000: | Loss: 0.02032 | Train Acc: 99.349 | Test Acc: 99.429
Epoch 000: | Loss: 0.02032 | Train Acc: 99.349 | Test Acc: 99.448
Epoch 000: | Loss: 0.02032 | Train Acc: 99.349 | Test Acc: 99.432
Epoch 000: | Loss: 0.02032 | Train Acc: 99.349 | Test Acc: 99.445
Epoch 000: | Loss: 0.02032 | Train Acc: 99.349 | Test Acc: 99.511
Epoch 000: | Loss: 0.02032 | Train Acc: 99.349 | Test Acc: 99.442
Epoch 000: | Loss: 0.02032 | Train Acc: 99.349 | Test Acc: 99.484
Epoch 000: | Loss: 0.02032 | Train Acc: 99.349 | Test Acc: 99.441
-----------------------------------------------------------------------------------------------------------------------

Epoch 001: | Loss: 0.01861 | Train Acc: 99.389 | Test Acc: 99.475
Epoch 001: | Loss: 0.01861 | Train Acc: 99.389 | Test Acc: 99.477
Epoch 001: | Loss: 0.01861 | Train Acc: 99.389 | Test Acc: 99.461
Epoch 001: | Loss: 0.01861 | Train Acc: 99.389 | Test Acc: 99.468
Epoch 001: | Loss: 0.01861 | Train Acc: 99.389 | Test Acc: 99.530
Epoch 001: | Loss: 0.

In [ ]:
save(y_test, "y_test")

In [ ]:
torch.save(model, "new-KNN.torch")

In [18]:
def get_count(preds):
    t = torch.round(torch.sigmoid(y_pred)).cpu().detach().numpy().reshape(-1)
    c1 = str(int(np.sum(t == 1 )))
    c0 = str(int(np.sum(t == 0 )))
    #print("1: "+ c1 + "| 2: "+ c2)
    return c1, c0

In [31]:
get_count(y_pred)

NameError: name 'y_pred' is not defined

In [28]:
pickle.dump(model, file)

In [29]:
file.close()

In [31]:
file = open('X_test.data', 'wb')
pickle.dump(X_test, file)
file.close()

In [33]:
file = open('y_test.data', 'wb')
pickle.dump(y_test, file)
file.close()

In [32]:
y_test

1856763    0
712274     0
1483074    0
101751     0
1853696    0
          ..
2442436    0
1149877    0
675851     0
876316     0
1791289    0
Name: Label, Length: 746167, dtype: int64

In [11]:
model

NameError: name 'model' is not defined

In [25]:
X_test

array([[0.        , 0.        , 0.        , ..., 0.08838295, 0.15467489,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.08838295, 0.15467489,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.1767659 , 0.13257847,
        0.5       ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.11047869, 0.08838565,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.15467017, 0.1767713 ,
        0.5       ]])

In [14]:
class tabularCNN(nn.Module):
    def __init__(self):
        super(tabularCNN, self).__init__()
        self.dense_1 = nn.Linear(111, 1024)
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm_1 = nn.BatchNorm1d(1024)
        self.batchnorm_2_1 = nn.BatchNorm1d(256)
        
        self.dense_repl = nn.Linear(1024, 8*6*6)
        self.batchnorm_repl = nn.BatchNorm1d(8*6*6)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.conv_1 = nn.Conv2d(1, 4, 3)
        self.conv_2 = nn.Conv2d(4, 8, 3)
        self.conv_3 = nn.Conv2d(8, 16, 3, stride=2)
        
        self.relu = nn.ReLU()
        
        self.dense_2_1 = nn.Linear(8*6*6, 256)
        self.dense_2_2 = nn.Linear(256, 1)
        
        
    def forward(self, inputs):
        x = self.relu(self.dense_1(inputs))
        x = self.batchnorm_1(x)
        x = self.dropout(x)
        
        x = self.relu(self.dense_repl(x))
        x = self.batchnorm_repl(x)
        x = self.dropout(x)
        
        #x = x.reshape(len(x), 1, 32, 32)
        #x = self.pool(self.relu(self.conv_1(x)))
        #x = self.pool(self.relu(self.conv_2(x)))

        #x = x.reshape(len(x), 8*6*6)
        
        x = self.relu(self.dense_2_1(x))
        x = self.batchnorm_2_1(x)
        x = self.dropout(x)
        
        x = self.relu(self.dense_2_2(x))    
        
        return x

In [15]:
#model = tabularCNN().to(device)
miodel = modelDNN().to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)
        
train_loader = DataLoader(dataset=train_data, batch_size=16384, shuffle=True)
epochs = 10

In [16]:
y_pred.shape[0] / X_test.shape[0]

NameError: name 'y_pred' is not defined

In [21]:
for e in range(epochs):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()

        y_pred = model(X_batch)

        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    model.eval()
    
    test_acc = 0
    for i in range(75):
        del y_pred
        y_pred = model(torch.Tensor(X_test[i*10000 : min(i*10000 + 10000, X_test.shape[0])]).to(device))
        c0, c1 = get_count(y_pred)
        test_acc += binary_acc(y_pred, torch.Tensor(np.array(y_test[i*10000 : min(i*10000 + 10000, y_test.shape[0])])).to(device).unsqueeze(1)) * (y_pred.shape[0] / X_test.shape[0])
    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Train Acc: {epoch_acc/len(train_loader):.3f} | Test Acc: {test_acc:.3f}')
    print("-----------------------------------------------------------------------------------------------------------------------\n")

tensor([[-8.7907],
        [-9.1293],
        [-7.8146],
        ...,
        [-8.6820],
        [-7.5472],
        [-8.1677]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], device='cuda:0', grad_fn=<RoundBackward0>)
tensor(16302., device='cuda:0')
tensor(0.9950, device='cuda:0')
---------------------------------
tensor([[-7.7492],
        [21.8608],
        [-8.3170],
        ...,
        [-8.0722],
        [-9.2845],
        [-8.7691]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], device='cuda:0', grad_fn=<RoundBackward0>)
tensor(16290., device='cuda:0')
tensor(0.9943, device='cuda:0')
---------------------------------
tensor([[-7.7287],
        [-6.4698],
        [-9.1703],
        ...,
        [26.0610],
        [-7.7646],
        [25.1712]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[

KeyboardInterrupt: 

In [21]:
model

tabularCNN(
  (dense_1): Linear(in_features=111, out_features=1024, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm_1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_1): Conv2d(1, 4, kernel_size=(5, 5), stride=(6, 6))
  (relu): ReLU()
  (dense_2): Linear(in_features=100, out_features=1, bias=True)
)

In [20]:
torch.save(model, "TabularCNN.torch")

In [38]:
input = torch.randn(20, 16, 50, 100)

In [39]:
m = nn.Conv2d(16, 33, 3, stride=2)

In [40]:
output = m(input)

In [41]:
output.shape

torch.Size([20, 33, 24, 49])